# DAY 1 - NOTEBOOK 3: PARTITIONING & CACHING

---

## 🎯 Learning Objectives:
1. Understand Partitioning concept
2. Master repartition() vs coalesce()
3. Learn Caching strategies
4. Understand Storage Levels
5. Apply best practices for performance

---

## SETUP: CREATE SPARK SESSION

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark import StorageLevel
import time

# Stop existing session
try:
    spark.stop()
    print("✅ Stopped existing session")
    time.sleep(2)
except:
    pass

# Create new session
spark = SparkSession.builder \
    .appName("Day1-PartitioningCaching") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "512m") \
    .config("spark.executor.cores", "1") \
    .config("spark.cores.max", "2") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

print("✅ Spark Session Created")
print(f"   App ID: {spark.sparkContext.applicationId}")
print(f"   Default Parallelism: {spark.sparkContext.defaultParallelism}")
print(f"   UI: http://localhost:4040")

---

## PART 1: UNDERSTANDING PARTITIONS

### 📚 Theory:

```
┌──────────────────────────────────────────────────────────────┐
│                    WHAT IS A PARTITION?                      │
├──────────────────────────────────────────────────────────────┤
│                                                              │
│  Partition = Một phần nhỏ của data                           │
│                                                              │
│  DataFrame with 1000 rows, 4 partitions:                     │
│                                                              │
│    ┌─────────────┐                                          │
│    │ Partition 0 │ → 250 rows → Worker 1, Core 1            │
│    ├─────────────┤                                          │
│    │ Partition 1 │ → 250 rows → Worker 1, Core 2            │
│    ├─────────────┤                                          │
│    │ Partition 2 │ → 250 rows → Worker 2, Core 1            │
│    ├─────────────┤                                          │
│    │ Partition 3 │ → 250 rows → Worker 2, Core 2            │
│    └─────────────┘                                          │
│                                                              │
│  Benefits:                                                   │
│    ✅ Parallel processing (4 tasks run simultaneously)       │
│    ✅ Distributed storage                                    │
│    ✅ Fault tolerance                                        │
│    ✅ Scalability                                            │
│                                                              │
└──────────────────────────────────────────────────────────────┘
```

### 🔬 Demo: Check Partitions

In [ ]:
# Create sample DataFrame
data = [(i, f"Name_{i}", i * 10) for i in range(1, 1001)]
df = spark.createDataFrame(data, ["id", "name", "value"])

print("📊 DataFrame Info:")
print(f"   Total rows: {df.count()}")
print(f"   Number of partitions: {df.rdd.getNumPartitions()}")
print(f"   Rows per partition (avg): {df.count() / df.rdd.getNumPartitions():.0f}")

### 🔬 Demo: Data Distribution Across Partitions

In [ ]:
print("📌 Data distribution across partitions:")
print("="*60)

# Get row count per partition
partition_counts = df.rdd.mapPartitionsWithIndex(
    lambda idx, iterator: [(idx, sum(1 for _ in iterator))]
).collect()

for partition_id, count in partition_counts:
    bar = "█" * (count // 10)
    print(f"Partition {partition_id}: {count:4d} rows {bar}")

print("="*60)

---

## PART 2: REPARTITION

### 📚 Theory:

```
repartition(n):
  • Tăng HOẶC giảm số partitions
  • Full shuffle (di chuyển data qua network)
  • Phân bổ data đều (balanced)
  • Chậm nhưng cân bằng
  • Use case: Tăng parallelism, balance data
```

In [ ]:
print("📌 Repartition: Increase partitions (2 → 8)")
print("="*60)

# Original partitions
print(f"Before: {df.rdd.getNumPartitions()} partitions")

# Repartition to 8
start = time.time()
df_8 = df.repartition(8)
df_8.count()  # Trigger action
elapsed = time.time() - start

print(f"After:  {df_8.rdd.getNumPartitions()} partitions")
print(f"Time:   {elapsed:.4f}s")
print("\n📊 Data distribution after repartition:")

partition_counts = df_8.rdd.mapPartitionsWithIndex(
    lambda idx, it: [(idx, sum(1 for _ in it))]
).collect()

for partition_id, count in partition_counts:
    bar = "█" * (count // 5)
    print(f"Partition {partition_id}: {count:4d} rows {bar}")

print("\n✅ Notice: Data is evenly distributed!")

### 🔬 Demo: Repartition by Column (Hash Partitioning)

In [ ]:
print("📌 Repartition by column (hash partitioning)")
print("="*60)

# Add department column
import random
random.seed(42)

data_with_dept = [
    (i, f"Name_{i}", i * 10, random.choice(["Eng", "Sales", "HR", "IT"]))
    for i in range(1, 1001)
]
df_dept = spark.createDataFrame(data_with_dept, ["id", "name", "value", "department"])

# Repartition by department
df_by_dept = df_dept.repartition(4, "department")

print(f"Repartitioned by 'department' into {df_by_dept.rdd.getNumPartitions()} partitions")
print("\n✅ Benefit: Same department data in same partition")
print("   → Faster groupBy operations on department")

---

## PART 3: COALESCE

### 📚 Theory:

```
coalesce(n):
  • CHỈ giảm số partitions (không tăng)
  • Minimal shuffle (ít di chuyển data)
  • Nhanh hơn repartition
  • Có thể không cân bằng (skewed)
  • Use case: Giảm partitions sau filter, trước write
```

In [ ]:
print("📌 Coalesce: Decrease partitions (8 → 2)")
print("="*60)

print(f"Before: {df_8.rdd.getNumPartitions()} partitions")

# Coalesce to 2
start = time.time()
df_2 = df_8.coalesce(2)
df_2.count()  # Trigger action
elapsed = time.time() - start

print(f"After:  {df_2.rdd.getNumPartitions()} partitions")
print(f"Time:   {elapsed:.4f}s")
print("\n📊 Data distribution after coalesce:")

partition_counts = df_2.rdd.mapPartitionsWithIndex(
    lambda idx, it: [(idx, sum(1 for _ in it))]
).collect()

for partition_id, count in partition_counts:
    bar = "█" * (count // 20)
    print(f"Partition {partition_id}: {count:4d} rows {bar}")

print("\n⚠️  Notice: May not be perfectly balanced (but faster!)")

---

## PART 4: REPARTITION vs COALESCE PERFORMANCE

In [ ]:
# Create larger dataset for better comparison
large_data = [(i, f"Name_{i}", i * 10) for i in range(1, 100001)]
df_large = spark.createDataFrame(large_data, ["id", "name", "value"]).repartition(20)

print(f"✅ Created large dataset: {df_large.count()} rows")
print(f"   Starting with {df_large.rdd.getNumPartitions()} partitions")

In [ ]:
print("\n🏁 Performance Test: repartition vs coalesce")
print("="*60)

# Test 1: repartition (20 → 4)
print("\n📌 Test 1: repartition(4) - Full shuffle")
start = time.time()
df_repart = df_large.repartition(4)
df_repart.count()
time_repart = time.time() - start
print(f"   Time: {time_repart:.4f}s")

# Test 2: coalesce (20 → 4)
print("\n📌 Test 2: coalesce(4) - Minimal shuffle")
start = time.time()
df_coal = df_large.coalesce(4)
df_coal.count()
time_coal = time.time() - start
print(f"   Time: {time_coal:.4f}s")

# Comparison
print("\n" + "="*60)
print("📊 COMPARISON:")
print(f"   repartition: {time_repart:.4f}s")
print(f"   coalesce:    {time_coal:.4f}s")
print(f"   ⚡ coalesce is {time_repart/time_coal:.2f}x faster!")
print("\n💡 Why? coalesce minimizes data movement (no full shuffle)")

---

## PART 5: WHEN TO USE REPARTITION vs COALESCE

```
┌──────────────────────────────────────────────────────────────┐
│                   USE REPARTITION WHEN:                      │
├──────────────────────────────────────────────────────────────┤
│  ✅ Tăng số partitions                                        │
│  ✅ Cần phân bổ data đều                                      │
│  ✅ Trước khi write nhiều files                               │
│  ✅ Trước khi join lớn                                        │
│  ✅ Partition by column (hash partitioning)                   │
│                                                              │
│  Examples:                                                   │
│    df.repartition(10).write.parquet("output/")              │
│    df.repartition(8, "department").groupBy("department")    │
└──────────────────────────────────────────────────────────────┘

┌──────────────────────────────────────────────────────────────┐
│                    USE COALESCE WHEN:                        │
├──────────────────────────────────────────────────────────────┤
│  ✅ Giảm số partitions                                        │
│  ✅ Sau khi filter (data nhỏ hơn)                             │
│  ✅ Trước khi collect                                         │
│  ✅ Trước khi write 1 file                                    │
│  ✅ Muốn tối ưu performance                                   │
│                                                              │
│  Examples:                                                   │
│    df.filter(...).coalesce(1).write.csv("output.csv")       │
│    df.filter(...).coalesce(2).collect()                     │
└──────────────────────────────────────────────────────────────┘
```

---

## PART 6: CACHING & PERSISTENCE

### 📚 Theory:

```
┌──────────────────────────────────────────────────────────────┐
│                    WHY CACHE?                                │
├──────────────────────────────────────────────────────────────┤
│                                                              │
│  Without Cache:                                              │
│    action1 → compute from source                             │
│    action2 → compute from source (again!)                    │
│    action3 → compute from source (again!)                    │
│                                                              │
│  With Cache:                                                 │
│    action1 → compute + store in memory                       │
│    action2 → read from memory (fast!)                        │
│    action3 → read from memory (fast!)                        │
│                                                              │
│  When to Cache:                                              │
│    ✅ DataFrame used multiple times                          │
│    ✅ Expensive computation (joins, aggregations)            │
│    ✅ Iterative algorithms (ML)                              │
│    ✅ Interactive analysis                                   │
│                                                              │
└──────────────────────────────────────────────────────────────┘
```

### 🔬 Demo: Cache Performance

In [ ]:
# Create test DataFrame with expensive transformation
df_test = spark.range(1, 1000000).toDF("id") \
    .withColumn("value", col("id") * 2) \
    .filter(col("id") % 2 == 0)

print("✅ Created test DataFrame with expensive transformation")

In [ ]:
print("📌 Test 1: WITHOUT cache (3 actions)")
print("="*60)

start = time.time()
count1 = df_test.count()  # Action 1: compute
count2 = df_test.count()  # Action 2: compute again!
count3 = df_test.count()  # Action 3: compute again!
time_no_cache = time.time() - start

print(f"✅ 3 actions completed")
print(f"   Time: {time_no_cache:.4f}s")
print(f"   Avg per action: {time_no_cache/3:.4f}s")
print("   ⚠️  Each action re-computed everything!")

In [ ]:
print("\n📌 Test 2: WITH cache (3 actions)")
print("="*60)

# Cache the DataFrame
df_cached = df_test.cache()

start = time.time()
count1 = df_cached.count()  # Action 1: compute + cache
count2 = df_cached.count()  # Action 2: use cache (fast!)
count3 = df_cached.count()  # Action 3: use cache (fast!)
time_with_cache = time.time() - start

print(f"✅ 3 actions completed")
print(f"   Time: {time_with_cache:.4f}s")
print(f"   Avg per action: {time_with_cache/3:.4f}s")
print("   ⚡ Used cache for 2nd and 3rd actions!")

# Comparison
print("\n" + "="*60)
print("📊 COMPARISON:")
print(f"   Without cache: {time_no_cache:.4f}s")
print(f"   With cache:    {time_with_cache:.4f}s")
print(f"   ⚡ Speedup: {time_no_cache/time_with_cache:.2f}x faster!")

# Clean up
df_cached.unpersist()
print("\n✅ Cache cleared")

---

## PART 7: STORAGE LEVELS

### 📚 Theory:

```
┌──────────────────────────────────────────────────────────────┐
│                   STORAGE LEVELS                             │
├──────────────────────────────────────────────────────────────┤
│                                                              │
│  MEMORY_ONLY (default for cache()):                          │
│    • Store in memory only                                    │
│    • Fast but limited by RAM                                 │
│    • If not fit, recompute                                   │
│    • Use: Small-medium datasets                              │
│                                                              │
│  MEMORY_AND_DISK:                                            │
│    • Store in memory first                                   │
│    • Spill to disk if not fit                                │
│    • Slower but more reliable                                │
│    • Use: Large datasets                                     │
│                                                              │
│  DISK_ONLY:                                                  │
│    • Store on disk only                                      │
│    • Slow but saves memory                                   │
│    • Use: Very large datasets, limited RAM                   │
│                                                              │
│  MEMORY_ONLY_SER:                                            │
│    • Serialize before storing                                │
│    • Save memory but slower                                  │
│    • Use: Memory-constrained environments                    │
│                                                              │
└──────────────────────────────────────────────────────────────┘
```

In [ ]:
print("📌 Storage Levels Demo")
print("="*60)

# MEMORY_ONLY (default)
print("\n1️⃣ MEMORY_ONLY (default)")
df_mem = df_test.cache()  # Same as persist(StorageLevel.MEMORY_ONLY)
df_mem.count()
print("   ✅ Cached in memory only")
df_mem.unpersist()

# MEMORY_AND_DISK
print("\n2️⃣ MEMORY_AND_DISK")
df_mem_disk = df_test.persist(StorageLevel.MEMORY_AND_DISK)
df_mem_disk.count()
print("   ✅ Cached in memory, spills to disk if needed")
df_mem_disk.unpersist()

# DISK_ONLY
print("\n3️⃣ DISK_ONLY")
df_disk = df_test.persist(StorageLevel.DISK_ONLY)
df_disk.count()
print("   ✅ Cached on disk only")
df_disk.unpersist()

print("\n✅ All storage levels demonstrated")

---

## PART 8: BEST PRACTICES

### 📋 Partitioning Best Practices:

```
1. Số partitions = 2-4x số cores
   • 2 workers × 2 cores = 4 cores
   • Recommended: 8-16 partitions

2. Partition size: 128MB - 1GB
   • Too small: overhead
   • Too large: memory issues

3. Repartition before:
   • Wide transformations (join, groupBy)
   • Writing to storage

4. Coalesce after:
   • Filter (reduced data)
   • Before collect
```

### 📋 Caching Best Practices:

```
1. Cache when:
   • DataFrame used multiple times
   • Expensive computation
   • Iterative algorithms

2. Don't cache when:
   • Used only once
   • Data too large for memory
   • Simple transformations

3. Always unpersist:
   • After done using
   • To free memory

4. Monitor:
   • Spark UI → Storage tab
   • Check memory usage
```

---

## PART 9: PRACTICAL EXAMPLE

In [ ]:
print("📌 Practical Example: Process large dataset with optimization")
print("="*60)

# Create large dataset
from faker import Faker
import random

fake = Faker()
Faker.seed(42)
random.seed(42)

large_data = []
for i in range(1, 10001):
    large_data.append({
        "id": i,
        "name": fake.name(),
        "department": random.choice(["Engineering", "Sales", "HR", "Marketing"]),
        "salary": random.randint(40000, 120000),
        "age": random.randint(22, 60)
    })

df_large = spark.createDataFrame(large_data)
print(f"✅ Created dataset: {df_large.count()} rows")
print(f"   Initial partitions: {df_large.rdd.getNumPartitions()}")

In [ ]:
print("\n🔹 Step 1: Repartition for parallel processing")
df_repart = df_large.repartition(8)
print(f"   Partitions: {df_repart.rdd.getNumPartitions()}")

In [ ]:
print("\n🔹 Step 2: Apply transformations")
df_transformed = df_repart \
    .filter(col("salary") > 50000) \
    .withColumn("annual_salary", col("salary") * 12) \
    .withColumn("age_group", 
        when(col("age") < 30, "Young")
        .when((col("age") >= 30) & (col("age") < 50), "Middle")
        .otherwise("Senior")
    )
print("   ✅ Transformations applied")

In [ ]:
print("\n🔹 Step 3: Cache for multiple operations")
df_cached = df_transformed.cache()
df_cached.count()  # Trigger caching
print("   ✅ Cached")

In [ ]:
print("\n🔹 Step 4: Multiple operations (use cache)")
start = time.time()

# Operation 1: Count by department
dept_count = df_cached.groupBy("department").count()
print("\n📊 Count by department:")
dept_count.show()

# Operation 2: Average salary by age group
age_salary = df_cached.groupBy("age_group").agg(avg("salary").alias("avg_salary"))
print("📊 Average salary by age group:")
age_salary.show()

# Operation 3: Top 10 highest paid
top_10 = df_cached.orderBy(col("salary").desc()).limit(10)
print("📊 Top 10 highest paid:")
top_10.show()

elapsed = time.time() - start
print(f"\n✅ All operations completed in {elapsed:.4f}s")
print("   ⚡ Fast because data was cached!")

In [ ]:
print("\n🔹 Step 5: Coalesce before final output")
df_final = df_cached.coalesce(2)
print(f"   Final partitions: {df_final.rdd.getNumPartitions()}")
print("   ✅ Ready for write/export")

In [ ]:
print("\n🔹 Step 6: Clean up")
df_cached.unpersist()
print("   ✅ Memory freed")

---

## ✅ SUMMARY

### 📚 What you learned:

**1️⃣ Partitioning:**
- What is a partition
- repartition() - full shuffle, balanced
- coalesce() - minimal shuffle, fast
- When to use each

**2️⃣ Caching:**
- Why cache (avoid recomputation)
- cache() vs persist()
- Storage levels (MEMORY_ONLY, MEMORY_AND_DISK, etc.)
- When to cache

**3️⃣ Performance:**
- Optimize partitions (2-4x cores)
- Cache for multiple actions
- Monitor Spark UI

**4️⃣ Best Practices:**
- Partition size: 128MB - 1GB
- Repartition before wide transformations
- Coalesce after filter
- Always unpersist when done

### 🎯 Key Takeaways:
- ✅ Understand partitioning for parallelism
- ✅ Use repartition for balance, coalesce for speed
- ✅ Cache expensive computations
- ✅ Choose right storage level
- ✅ Always clean up (unpersist)

### 🎉 DAY 1 COMPLETED!

**You now understand:**
- ✅ Spark Architecture
- ✅ DataFrame API
- ✅ Transformations vs Actions
- ✅ Lazy Evaluation
- ✅ Partitioning strategies
- ✅ Caching techniques

### 📝 Next: Day 2 - Data I/O & Cleaning
- Reading/Writing different formats
- Data cleaning techniques
- Handling missing data
- Type conversions